# ENDEPORTE_PROYECTO_FINAL

### Objetivo:
Mediante tecnicas de machine learning establecer las variables que mayor relacion tienen con la desercion e identificar los estudiantes de carreras profesionales o tecnologicas de la entidad que pueden desertar o no de sus programas.

### Contexto del negocio
Para la entidad es dificil identificar tempranamente los estudiantes que están en riesgo de abandonar sus carreras, las intervenciones son a menudo menos efectivas porque las señales de riesgo no aparecen hasta que el estudiante ya ha tomado la decisión de desertar. Se espera que mediante modelos de machine learning se identifiquen con anticipacion los estudiantes que pueden llegar a desertar con el fin de brindar a cada estudiante el tipo y nivel de apoyo que necesita para superar sus dificultades.

In [2]:
!pip install scikit-learn

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import unicodedata
import warnings

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
from sklearn.preprocessing import LabelEncoder # Import LabelEncoder



##**1. Entendimiento y preparación de los datos:**


### Cargue del dataset

In [6]:
demograficos_url="https://github.com/amrios80/Endeporte/raw/refs/heads/main/data/Demograficos.csv.gz"
historicos_url="https://github.com/amrios80/Endeporte/raw/refs/heads/main/data/HistoricoNotas.csv.gz"

demograficos_df = pd.read_csv(demograficos_url, sep=";", compression='gzip')
historicos_df=pd.read_csv(historicos_url, sep=";", compression="gzip")

In [7]:
divipola = pd.read_csv('../raw/DIVIPOLA-_C_digos_municipios_20241127.csv', sep=",", encoding='utf-8')

### Limpieza de datos

* ***Estandarizacion*** : Segun las reglas acordadas con ENDEPORTE, Se excluyen del estudio las ESPECIALIZACIONES:

In [10]:
demograficos_df.sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
50897,2024 - 1,104024231106,ESCUELA NACIONAL DEL DEPORTE,104024,TERAPIA OCUPACIONAL,3,"3,6",Femenino,10-02-2004,1.0,CALI,VALLE DEL CAUCA,Soltero,"3,7",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,03/01/24,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2024-I,20,Edad entre 18 y 25 años,Fac. Salud,Universitrario
20512,2019 - 2T,7305192T190,ESCUELA NACIONAL DEL DEPORTE,7305,TECNOLOGIA EN DEPORTE,1,"3,2",Masculino,21-12-1990,3.0,CALI,VALLE DEL CAUCA,Soltero,"3,2",BAJO RENDIMIENTO ACADEMICO,ANTIGUO,BAJO RENDIMIENTO ACADEMICO,ANTIGUO,NINGUNA,NINGUNA,29/08/19,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2019-II,29,Edad entre 26 y 33 años,Fac. Deporte,Tecnológico
47929,2023 - 2,102383202091,ESCUELA NACIONAL DEL DEPORTE,102383,NUTRICION Y DIETETICA,7,"3,9",Femenino,08-05-2003,3.0,TUMACO,NARIÑO,Soltero,"4,2",ACTIVO,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,05/07/23,52480.0,NARIÑO,52.0,NARIÑO,1.0,1.0,1,2023-II,20,Edad entre 18 y 25 años,Fac. Salud,Universitrario


In [11]:
demograficos_df["NIVEL_FORMACION"].value_counts()

NIVEL_FORMACION
Universitrario     48022
Tecnológico         4994
Especialización     2403
Name: count, dtype: int64

In [12]:
demograficos_df=demograficos_df[(demograficos_df["NIVEL_FORMACION"] != "Especialización")]



*  ***Eliminacion de codigos de estudiantes duplicados*** : La informacion demografica debe estar a nivel de estudiante, segun lo definido con la entidad, se valida que no haya duplicidad a nivel de codigo del estudiante.



In [14]:
total_rows = demograficos_df.shape[0]
total_rows

53016

In [15]:
demograficos_df.loc[demograficos_df.duplicated(subset=["CODIGO"], keep=False)].sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
44952,2023 - 1,102383162208,ESCUELA NACIONAL DEL DEPORTE,102383,NUTRICION Y DIETETICA,8,"3,2",Femenino,01-06-1998,1.0,CALOTO,CAUCA,Soltero,"3,4",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,15/01/23,19698.0,SANTANDER DE QUILICHAO,19.0,CAUCA,1.0,1.0,1,2023-I,25,Edad entre 18 y 25 años,Fac. Salud,Universitrario
22054,2020 - 1,104024171027,ESCUELA NACIONAL DEL DEPORTE,104024,TERAPIA OCUPACIONAL,7,"4,6",Masculino,25-03-1999,2.0,VIJES,VALLE DEL CAUCA,Soltero,"4,1",EXCLUIDO CANCELACION SEMESTRE,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,27/01/20,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2020-I,21,Edad entre 18 y 25 años,Fac. Salud,Universitrario
43200,2022 - 2,90555222130,ESCUELA NACIONAL DEL DEPORTE,90555,ADMINISTRACION DE EMPRESAS,1,"4,7",Masculino,19-10-2000,2.0,CALI,VALLE DEL CAUCA,Soltero,"4,7",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,27/07/22,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2022-II,22,Edad entre 18 y 25 años,Fac. Admon,Universitrario


In [16]:
demograficos_df[demograficos_df["CODIGO"]=="102383181022"].sort_values(by="PERIODO_ACAENICO",ascending=False).head(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION
38163,2022 - 1,102383181022,ESCUELA NACIONAL DEL DEPORTE,102383,NUTRICION Y DIETETICA,7,"4,6",Femenino,06-04-2000,3.0,CALI,VALLE DEL CAUCA,Soltero,"4,2",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,16/02/22,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2022-I,22,Edad entre 18 y 25 años,Fac. Salud,Universitrario
34325,2021 - 2,102383181022,ESCUELA NACIONAL DEL DEPORTE,102383,NUTRICION Y DIETETICA,8,"4,2",Femenino,06-04-2000,3.0,CALI,VALLE DEL CAUCA,Soltero,"4,1",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,30/07/21,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-II,21,Edad entre 18 y 25 años,Fac. Salud,Universitrario
30500,2021 - 1,102383181022,ESCUELA NACIONAL DEL DEPORTE,102383,NUTRICION Y DIETETICA,7,"4,4",Femenino,06-04-2000,3.0,CALI,VALLE DEL CAUCA,Soltero,"3,9",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,12/01/21,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-I,21,Edad entre 18 y 25 años,Fac. Salud,Universitrario


Se observa que el codigo de estudiante se repite tantas veces como periodos académicos haya cursado y las variables de semestre, promediosemestre, promediogeneral y fecha_matricula son las correspondientes al periodo academico, las demas variables demograficas son las mismas para todos los periodos academicos. Por lo anterior, se tomara la informacion demografica correspondiente al ultimo periodo academico.

In [18]:
idx = demograficos_df.groupby('CODIGO')['PERIODO_ACAENICO'].idxmax()

In [19]:
max_periodo_rows = demograficos_df.loc[idx]

In [20]:
demograficos_df_uniq=max_periodo_rows.copy()

In [21]:
total_rows_uniq = demograficos_df_uniq.shape[0]
total_rows_uniq

10021

El estudio se realizara con la informacion demografica correspondiente al ultimo periodo academico de 10.021 estudiantes unicos.

*  ***Calculo de la variable objetivo (Desercion)*** : Si la variable denominada “situación actual” contiene cumple las siguientes condiciones “excluido cancelación semestre” o “excluido no renovación matricula” o “retiro definitivo” y la variable denominada “categoría actual” el valor es diferente de “continuidad académica - egresado” se considera “Deserción”.

In [24]:
def calculate_target(row):
    if (row['SITUACION_ACTUAL'] == 'EXCLUIDO NO RENOVACION DE MATRICULA' or row["SITUACION_ACTUAL"]=='RETIRO DEFINITIVO' or row["SITUACION_ACTUAL"]=='EXCLUIDO CANCELACION SEMESTRE' ) and  (row['CATEGORIA_ACTUAL'] != "CONTINUIDAD ACADEMICA - EGRESADO") :
        return 0
    else:
        return 1

In [25]:
demograficos_df_uniq["target"]=demograficos_df_uniq.apply(calculate_target, axis=1)

In [26]:
demograficos_df_uniq["target"].value_counts()

target
1    8760
0    1261
Name: count, dtype: int64

Se observa que 1.261 estudiantes de 10.021 desertan, es decir la tasa de desercion de la poblacion estudiada es de 12,6% valor que coincide con las cifras de desercion reportadas por la entidad.

* ***Creacion del dataset (features + target) :*** De acuerdo al analisis exploratorio desarrollado para la primera entrega, se tendran en cuenta para el estudio las siguientes variables, dada su relacion directa con la desercion (target): ESTRATO, SEMESTRE, PROMEDIOSEMESTRE, EDAD2, PROGRAMA, GENERO, CIUDADRESIDENCIA, CIUDADNACIMIENTO. Con base en estas variables de entrada se creara el dataset previa conversion numerica de las variables: ESTRATO, SEMESTRE, PROMEDIOSEMESTRE, EDAD2 y codificacion de las variables categoricas: PROGRAMA, GENERO, CIUDADRESIDENCIA, CIUDADNACIMIENTO.

Conversion a valor numerico del ESTRATO, SEMESTRE, PROMEDIOSEMESTRE Y EDAD2

In [30]:
field="ESTRATO"
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field], errors='coerce')
field="SEMESTRE"
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field], errors='coerce')
field="PROMEDIOSEMESTRE"
demograficos_df_uniq[field] = demograficos_df_uniq[field].astype(str).str.replace(',', '.', regex=False)
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field], errors='coerce')
field="EDAD2"
demograficos_df_uniq[field] = pd.to_numeric(demograficos_df_uniq[field], errors='coerce')

In [31]:
demograficos_df_uniq.sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION,target
47486,2023 - 2,53212202199,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,6,3.9,Femenino,30-03-2001,2.0,CALI,VALLE DEL CAUCA,Soltero,4,ACTIVO,REINGRESO,ACTIVO,ANTIGUO,DISCAPACIDAD AUDITIVA,DISCAPACIDAD SENSORIAL - HIPOACUSIA,04/07/23,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2023-II,22,Edad entre 18 y 25 años,Fac. Deporte,Universitrario,1
32861,2021 - 2,102383172147,ESCUELA NACIONAL DEL DEPORTE,102383,NUTRICION Y DIETETICA,7,4.2,Femenino,03-04-1999,3.0,SANTANDER DE QUILICHAO,CAUCA,Soltero,"3,7",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,15/07/21,19698.0,SANTANDER DE QUILICHAO,19.0,CAUCA,0.0,NaN,1,2021-II,22,Edad entre 18 y 25 años,Fac. Salud,Universitrario,1
46081,2023 - 1,53212182127,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,8,3.9,Masculino,20-08-2000,1.0,CALI,VALLE DEL CAUCA,Soltero,"3,6",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,23/12/22,52473.0,MOSQUERA,52.0,NARIÑO,1.0,1.0,1,2023-I,22,Edad entre 18 y 25 años,Fac. Deporte,Universitrario,1


In [32]:
# Función para eliminar tildes de los textos
def quitar_tildes(texto):
    # Asegurarse de que el texto sea una cadena
    if isinstance(texto, str):
        return ''.join(
            c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'
        )
    return texto  # Si no es una cadena, retornar el valor tal cual (puede ser NaN, float, etc.)


In [33]:
# Aplicar la función de quitar tildes a las columnas relevantes de ambos DataFrames
demograficos_df_uniq['CIUDADRESIDENCIA'] = demograficos_df_uniq['CIUDADRESIDENCIA'].apply(quitar_tildes)
divipola['Nombre_Municipio'] = divipola['Nombre_Municipio'].apply(quitar_tildes)

In [34]:
relacion = pd.merge(demograficos_df_uniq, 
                    divipola[['Codigo_Municipio','Nombre_Municipio']], 
                    left_on='CIUDADRESIDENCIA', 
                    right_on='Nombre_Municipio', 
                    how='left').rename(
    columns={ 
        'Codigo_Municipio': 'COD_CIUDADRESIDENCIA', 
        'Nombre_Municipio': 'Nombre_CIUDADRESIDENCIA'}
)

relacion = pd.merge(relacion, 
                    divipola[['Codigo_Municipio','Nombre_Municipio']], 
                    left_on='IDCIUDADNACIMIENTO', 
                    right_on='Codigo_Municipio', 
                    how='left').rename(
    columns={ 
        'Codigo_Municipio': 'COD_CIUDADNACIMIENTO', 
        'Nombre_Municipio': 'Nombre_CIUDADNACIMIENTO'}
)
relacion.sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION,target,COD_CIUDADRESIDENCIA,Nombre_CIUDADRESIDENCIA,COD_CIUDADNACIMIENTO,Nombre_CIUDADNACIMIENTO
5444,2024 - 1,53212231301,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,3,0.4,Masculino,30-06-1998,3.0,CALI,VALLE DEL CAUCA,Soltero,3,BAJO RENDIMIENTO ACADEMICO,ANTIGUO,BAJO RENDIMIENTO ACADEMICO,ANTIGUO,NINGUNA,NINGUNA,02/01/24,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2024-I,26,Edad entre 26 y 33 años,Fac. Deporte,Universitrario,1,76001.0,CALI,76001.0,CALI
3870,2021 - 1,53212172344,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,8,4.8,Femenino,15-09-1995,3.0,CALI,VALLE DEL CAUCA,Soltero,"4,4",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,28/01/21,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-I,26,Edad entre 26 y 33 años,Fac. Deporte,Universitrario,1,76001.0,CALI,76001.0,CALI
6696,2024 - 1G,7305221G027,CERES SALTO AFRO GUACHENE CAUCA,7305,TECNOLOGIA EN DEPORTE,5,3.7,Masculino,19-01-2000,3.0,CANDELARIA,VALLE DEL CAUCA,Soltero,"3,5",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,01/02/24,76001.0,CALI,76.0,VALLE DEL CAUCA,1.0,1.0,1,2024-I,24,Edad entre 18 y 25 años,Fac. Deporte,Tecnológico,1,8141.0,CANDELARIA,76001.0,CALI


In [35]:
# Revision de cruces
relacion[relacion['CIUDADRESIDENCIA']=='GUACHENE'].sample(3)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION,target,COD_CIUDADRESIDENCIA,Nombre_CIUDADRESIDENCIA,COD_CIUDADNACIMIENTO,Nombre_CIUDADNACIMIENTO
2068,2024 - 1,110783232092,ESCUELA NACIONAL DEL DEPORTE,110783,MERCADEO Y NEGOCIOS INTERNACIONALES,2,4.1,Masculino,16-05-2006,1.0,GUACHENE,CAUCA,Soltero,"4,1",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,02/01/24,19300.0,GUACHENE,19.0,CAUCA,0.0,NaN,1,2024-I,18,Edad entre 18 y 25 años,Fac. Admon,Universitrario,1,19300.0,GUACHENE,19300.0,GUACHENE
6186,2018 - 1T,7305172T316,ESCUELA NACIONAL DEL DEPORTE,7305,TECNOLOGIA EN DEPORTE,2,3.0,Masculino,06-06-1998,1.0,GUACHENE,CAUCA,Soltero,"3,4",EXCLUIDO NO RENOVACION DE MATRICULA,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,15/01/18,19300.0,GUACHENE,19.0,CAUCA,1.0,1.0,1,2018-I,20,Edad entre 18 y 25 años,Fac. Deporte,Tecnológico,0,19300.0,GUACHENE,19300.0,GUACHENE
5584,2024 - 1,53212232439,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,2,3.9,Masculino,20-09-2005,1.0,GUACHENE,CAUCA,Soltero,"3,6",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,03/01/24,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2024-I,19,Edad entre 18 y 25 años,Fac. Deporte,Universitrario,1,19300.0,GUACHENE,76001.0,CALI


In [36]:
# Revision de Genero
relacion['GENERO'].unique()

array(['Masculino', 'Femenino'], dtype=object)

In [37]:
# Mapeo de valores de genero
genero_map = {'Masculino': 1, 'Femenino': 0}

In [38]:
relacion['GENERO'] = relacion['GENERO'].str.strip() # Eliminar espacios
relacion['GENERO'] = relacion['GENERO'].str.title()  # Convertir a formato de título (Primera letra en mayúscula)

In [39]:
# MAPEO DE VALORES NUMERICOS DE GENERO
relacion['GENERO'] = relacion['GENERO'].map(genero_map)
relacion.sample(5)

,PERIODO_ACAENICO,CODIGO,UNIDAD,CODIGOPROGRAMA,PROGRAMA,SEMESTRE,PROMEDIOSEMESTRE,GENERO,FECHANACIMIENTO,ESTRATO,CIUDADRESIDENCIA,DEPARTAMENTORESICEDENCIA,ESTADOCIVIL,PROMEDIOGENERAL,SITUACION_ACTUAL,CATEGORIA_ACTUAL,SITUACION_DEL_PERIODO,CATEGORIA_DEL_PERIODO,DISCAPACIDAD,TIPO_DISCAPACIDAD,FECHA_MATRICULA,IDCIUDADNACIMIENTO,CIUDADNACIMIENTO,IDDEPARTAMENTONACIMIENTO,DEPARTAMENTONACIMIENTO,SISBEN,NIVELSISBEN,CANTIDAD,PERIODO,EDAD2,EDAD,FACULTAD,NIVEL_FORMACION,target,COD_CIUDADRESIDENCIA,Nombre_CIUDADRESIDENCIA,COD_CIUDADNACIMIENTO,Nombre_CIUDADNACIMIENTO
3532,2021 - 2,53212171015,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,8,4.0,0,09-10-1999,4.0,CALI,VALLE DEL CAUCA,Soltero,"3,8",EXCLUIDO NO RENOVACION DE MATRICULA,CONTINUIDAD ACADEMICA - EGRESADO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,29/07/21,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2021-II,22,Edad entre 18 y 25 años,Fac. Deporte,Universitrario,1,76001.0,CALI,76001.0,CALI
9978,2024 - 1,90555212192,ESCUELA NACIONAL DEL DEPORTE,90555,ADMINISTRACION DE EMPRESAS,4,3.2,0,09-09-2000,2.0,CALI,VALLE DEL CAUCA,Soltero,"3,2",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,13/01/24,76001.0,CALI,76.0,VALLE DEL CAUCA,1.0,1.0,1,2024-I,24,Edad entre 18 y 25 años,Fac. Admon,Universitrario,1,76001.0,CALI,76001.0,CALI
9052,2024 - 1,7306232267,ESCUELA NACIONAL DEL DEPORTE,7306,FISIOTERAPIA,2,3.7,0,13-12-2003,3.0,CALI,VALLE DEL CAUCA,Soltero,"3,8",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,02/01/24,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2024-I,21,Edad entre 18 y 25 años,Fac. Salud,Universitrario,1,76001.0,CALI,76001.0,CALI
8686,2024 - 1,7306211334,ESCUELA NACIONAL DEL DEPORTE,7306,FISIOTERAPIA,7,3.7,0,13-04-2003,1.0,PRADERA,VALLE DEL CAUCA,Soltero,"3,9",ACTIVO,ANTIGUO,ACTIVO,ANTIGUO,NINGUNA,NINGUNA,02/01/24,11001.0,BOGOTA D.C.,11.0,"BOGOTÁ, DISTRITO CAPITAL",0.0,NaN,1,2024-I,21,Edad entre 18 y 25 años,Fac. Salud,Universitrario,1,76563.0,PRADERA,11001.0,BOGOTA. D.C.
2996,2017 - 1,53212151291,ESCUELA NACIONAL DEL DEPORTE,53212,DEPORTE,5,2.9,1,14-03-1995,2.0,CALI,VALLE DEL CAUCA,Soltero,"3,3",EXCLUIDO NO RENOVACION DE MATRICULA,ANTIGUO,BAJO RENDIMIENTO ACADEMICO,ANTIGUO,NINGUNA,NINGUNA,11/01/17,76001.0,CALI,76.0,VALLE DEL CAUCA,0.0,NaN,1,2017-I,22,Edad entre 18 y 25 años,Fac. Deporte,Universitrario,0,76001.0,CALI,76001.0,CALI


In [40]:
numeric_vars = ["ESTRATO", "SEMESTRE", "PROMEDIOSEMESTRE", "EDAD2",'CODIGOPROGRAMA',"GENERO","COD_CIUDADRESIDENCIA",'COD_CIUDADNACIMIENTO']
#category_vars = ["PROGRAMA", "GENERO", "CIUDADRESIDENCIA","CIUDADNACIMIENTO"]
category_vars =[]
target_var="target"
features=numeric_vars + category_vars
features

['ESTRATO',
 'SEMESTRE',
 'PROMEDIOSEMESTRE',
 'EDAD2',
 'CODIGOPROGRAMA',
 'GENERO',
 'COD_CIUDADRESIDENCIA',
 'COD_CIUDADNACIMIENTO']

Conversion de las variables categoricas a numericas utilizando codificacion, para asegurar que todas las variables de entrada al modelo sean numericas.

In [42]:
def get_df(df, target_var="target", numeric_vars=[], category_vars=[]):
  df = df.copy()
  label_encoder = LabelEncoder()
  for col in category_vars:
      df[col] = label_encoder.fit_transform(df[col].astype(str))
  df=df[ numeric_vars + category_vars+ [target_var]]
  return df

In [43]:
df=relacion
df=get_df(df, target_var, numeric_vars, category_vars)
df.sample(3)

,ESTRATO,SEMESTRE,PROMEDIOSEMESTRE,EDAD2,CODIGOPROGRAMA,GENERO,COD_CIUDADRESIDENCIA,COD_CIUDADNACIMIENTO,target
10206,3.0,2,4.0,24,90555,0,76001.0,76001.0,1
3450,2.0,8,4.1,22,53212,1,19142.0,19001.0,1
10232,3.0,2,4.3,56,90555,0,76001.0,76001.0,1


* ***Identificacion valores nulos en las variables de entrada:***

In [45]:
df[[target_var]+features].isnull().sum() / df.shape[0]

target                  0.000000
ESTRATO                 0.000578
SEMESTRE                0.000000
PROMEDIOSEMESTRE        0.000000
EDAD2                   0.000000
CODIGOPROGRAMA          0.000000
GENERO                  0.000000
COD_CIUDADRESIDENCIA    0.035171
COD_CIUDADNACIMIENTO    0.006456
dtype: float64

Se observa que pocas columnas del dataset tiene valores nulos, sin embargo representan un porcentaje minimo, por lo anterior decidimos conservar todas las variables de entrada.

In [47]:
duplicated_rows = df.loc[df.duplicated(subset=features, keep=False)].shape[0]
duplicated_rows

1462

In [48]:
print(f"Duplicates: {(duplicated_rows/total_rows)*100:.4f}%")

Duplicates: 2.7577%


In [49]:
duplicated_rows = df.loc[df.duplicated(subset=features+["target"], keep=False)].shape[0]
duplicated_rows

1385

In [50]:
print(f"Duplicates: {(duplicated_rows/total_rows)*100:.4f}%")

Duplicates: 2.6124%




Existe un ~2.75% de registros con variables de entrada duplicadas, incluso con variable objetivo diferente. Al incluir la variable objetivo dentro del análisis de duplicados, se obtiene el ~2.61% registros duplicados adicionales. Para un porcentaje total de 5,36%, se decide eliminar los datos duplicados con el fin de no afectar en la funcion de estimacion.


In [52]:
df.drop_duplicates(subset=features, inplace=True)
df.drop_duplicates(subset=features+["target"], inplace=True)

In [53]:
df.shape

(9453, 9)

In [54]:
df.to_csv('../../data/stage/clean.csv', index=False)